# 🔗 Chains: Conectando o Mundo das LLMs como um Elo Perdido!

**Módulo 6 - LangChain v0.3**

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-06_img_01.png)

Bora entender o que são **Chains** no LangChain! 🚀

Tá, mas o que é uma Chain? Imagina que você tá fazendo um brigadeiro... você não vai direto do leite condensado pro brigadeiro pronto, né? Você tem etapas: misturar, mexer, resfriar, fazer as bolinhas...

**Chain** é exatamente isso: uma sequência de etapas conectadas onde o resultado de uma vira entrada da próxima!

No LangChain, conectamos **ChatModels**, **PromptTemplates**, **OutputParsers** (que já vimos nos módulos anteriores) numa sequência liiinda! ✨

## 🎯 O que vamos aprender?

- **Conceito fundamental** de Chains
- **LangChain Expression Language (LCEL)** - nossa linguagem do amor com pipes (`|`)
- **Chain simples** com ChatModel + PromptTemplate
- **Chain completa** com PromptTemplate + ChatModel + OutputParser
- **Chains sequenciais** - uma Chain chamando outra!
- **Debugging** e monitoramento de Chains
- Casos práticos que vão te fazer falar "Liiindo!"

**Dica!** Chains são a base de TUDO no LangChain. Dominar isso é meio caminho andado pro resto do curso! 🎪

In [ ]:
# Bora começar instalando e importando o que precisamos!
!pip install langchain langchain-google-genai python-dotenv -q

import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente
load_dotenv()

# Se não tiver o .env, descomenta e coloca sua chave aqui:
# os.environ["GOOGLE_API_KEY"] = "sua_chave_aqui"

print("✅ Ambiente configurado!")

In [ ]:
# Imports necessários para nosso show de Chains
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("📦 Pacotes importados com sucesso!")
print("Agora sim, bora criar nossas primeiras Chains! 🔗")

## 🧠 Entendendo o Conceito de Chain

Antes de partir pro código, vamos entender a **filosofia** por trás das Chains.

### A Analogia da Fábrica de Pão de Açúcar 🏭

Imagina uma fábrica de pão de açúcar:

1. **Entrada**: Farinha, ovos, açúcar (seus dados brutos)
2. **Estação 1**: Misturador (PromptTemplate - organiza os ingredientes)
3. **Estação 2**: Forno (ChatModel - processa e "assa" a informação)
4. **Estação 3**: Empacotador (OutputParser - formata o resultado final)
5. **Saída**: Pão de açúcar prontinho!

### Fórmula Matemática de uma Chain:

$$Chain(x) = f_3(f_2(f_1(x)))$$

Onde:
- $f_1$ = PromptTemplate
- $f_2$ = ChatModel 
- $f_3$ = OutputParser
- $x$ = Input inicial

**Dica!** No LangChain, isso fica assim: `chain = prompt | model | parser` 🎯

## 🔧 Configurando Nossos Componentes Básicos

Antes de fazer a Chain, vamos preparar cada "peça" separadamente. Lembra dos módulos anteriores? Agora vamos conectar tudo!

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-06_img_02.png)

In [ ]:
# Criando nossos componentes básicos

# 1. ChatModel - nosso processador principal
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.7
)

# 2. PromptTemplate - nosso organizador de dados
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente prestativo que explica conceitos de forma simples."),
    ("user", "Explique o conceito de {conceito} em no máximo 100 palavras.")
])

# 3. OutputParser - nosso formatador final
parser = StrOutputParser()

print("🔧 Componentes criados!")
print(f"Model: {model.model_name}")
print(f"Prompt: Configurado com template para conceitos")
print(f"Parser: {type(parser).__name__}")

## 🚀 Nossa Primeira Chain Simples

Agora a mágica acontece! Vamos criar nossa primeira Chain usando o **LCEL** (LangChain Expression Language).

O operador `|` (pipe) é nosso melhor amigo aqui. Ele conecta os componentes como se fosse um cano d'água levando informação de um lugar pro outro!

In [ ]:
# 🔗 Criando nossa primeira Chain!
# A sintaxe é MUITO simples: componente1 | componente2 | componente3

chain_simples = prompt | model | parser

print("🎉 Chain criada com sucesso!")
print("Estrutura da Chain: PromptTemplate -> ChatModel -> OutputParser")
print("\nTipo da chain:", type(chain_simples))

# Vamos testar nossa Chain!
resultado = chain_simples.invoke({"conceito": "inteligência artificial"})

print("\n" + "="*50)
print("🤖 RESULTADO DA CHAIN:")
print("="*50)
print(resultado)

## 📊 Visualizando o Fluxo da Chain

Vamos criar um diagrama pra entender melhor o que tá acontecendo por baixo dos panos:

In [ ]:
# Vamos "espiar" o que acontece em cada etapa da Chain
import json

def debug_chain_step_by_step(conceito_teste):
    """Executa a chain passo a passo para debug"""
    
    input_data = {"conceito": conceito_teste}
    
    print("🔍 DEBUG DA CHAIN - PASSO A PASSO")
    print("="*60)
    
    # Passo 1: PromptTemplate
    print("\n1️⃣ ENTRADA:")
    print(f"   Input: {input_data}")
    
    # Passo 2: Prompt processado
    prompt_result = prompt.invoke(input_data)
    print("\n2️⃣ PROMPT TEMPLATE (formatando):")
    print(f"   Prompt formatado: {prompt_result.messages[1].content}")
    
    # Passo 3: Model processando
    print("\n3️⃣ CHAT MODEL (processando)...")
    model_result = model.invoke(prompt_result)
    print(f"   Resposta bruta do modelo: {model_result.content[:100]}...")
    
    # Passo 4: Parser finalizando
    final_result = parser.invoke(model_result)
    print("\n4️⃣ OUTPUT PARSER (finalizando):")
    print(f"   Resultado final: {final_result}")
    
    return final_result

# Testando nosso debug
debug_chain_step_by_step("machine learning")

## 🎭 Chains com Diferentes Tipos de Output

Tá, mas e se eu quiser um resultado estruturado? Lembra do módulo de **OutputParsers**? Bora usar aqui!

Vamos criar uma Chain que retorna dados estruturados sobre tecnologias:

In [ ]:
# Importando parser estruturado que vimos no módulo anterior
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Definindo a estrutura que queremos
class TecnologiaInfo(BaseModel):
    nome: str = Field(description="Nome da tecnologia")
    descricao: str = Field(description="Descrição em português simples")
    dificuldade: str = Field(description="Iniciante, Intermediário ou Avançado")
    areas_uso: list = Field(description="Lista de áreas onde é usada")

# Criando parser JSON
json_parser = JsonOutputParser(pydantic_object=TecnologiaInfo)

# Novo prompt para dados estruturados
prompt_estruturado = ChatPromptTemplate.from_messages([
    ("system", "Você é um expert em tecnologia que retorna informações estruturadas.\n{format_instructions}"),
    ("user", "Me dê informações sobre: {tecnologia}")
])

print("🏗️ Componentes para Chain estruturada criados!")

In [ ]:
# Criando nossa Chain estruturada
chain_estruturada = (
    {
        "tecnologia": RunnablePassthrough(),
        "format_instructions": lambda x: json_parser.get_format_instructions()
    }
    | prompt_estruturado
    | model
    | json_parser
)

print("🔗 Chain estruturada criada!")

# Testando com diferentes tecnologias
tecnologias_teste = ["Python", "React", "Docker"]

for tech in tecnologias_teste:
    print(f"\n📋 Informações sobre {tech}:")
    print("-" * 40)
    
    resultado = chain_estruturada.invoke(tech)
    
    print(f"Nome: {resultado['nome']}")
    print(f"Descrição: {resultado['descricao']}")
    print(f"Dificuldade: {resultado['dificuldade']}")
    print(f"Áreas de uso: {', '.join(resultado['areas_uso'])}")

## 🔄 Chains Sequenciais - Uma Chain Chamando Outra!

Agora vamos pro próximo nível! E se eu quiser que o resultado de uma Chain vire entrada de outra Chain?

Tipo assim: **Chain 1** gera uma ideia → **Chain 2** desenvolve a ideia → **Chain 3** critica a ideia

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-06_img_03.png)

In [ ]:
# Vamos criar um sistema de 3 Chains:
# Chain 1: Gera uma ideia de startup
# Chain 2: Desenvolve a ideia
# Chain 3: Faz uma crítica construtiva

# Chain 1: Gerador de ideias
prompt_gerador = ChatPromptTemplate.from_messages([
    ("system", "Você é um gerador criativo de ideias de startup."),
    ("user", "Gere uma ideia inovadora de startup na área de {area}. Seja criativo mas realista!")
])

chain_gerador = prompt_gerador | model | StrOutputParser()

# Chain 2: Desenvolvedor de ideias
prompt_desenvolvedor = ChatPromptTemplate.from_messages([
    ("system", "Você é um consultor de negócios experiente."),
    ("user", "Desenvolva esta ideia de startup com mais detalhes, incluindo modelo de negócio e público-alvo:\n\n{ideia}")
])

chain_desenvolvedor = prompt_desenvolvedor | model | StrOutputParser()

# Chain 3: Crítico construtivo
prompt_critico = ChatPromptTemplate.from_messages([
    ("system", "Você é um investidor experiente que faz análises críticas mas construtivas."),
    ("user", "Analise esta proposta de startup e dê feedback honesto sobre pontos fortes, fracos e sugestões:\n\n{proposta}")
])

chain_critico = prompt_critico | model | StrOutputParser()

print("🏭 Três chains criadas para nosso pipeline de análise de startup!")

In [ ]:
# Agora vamos conectar as 3 chains em sequência!

def pipeline_completo_startup(area):
    """Executa o pipeline completo: gera -> desenvolve -> critica"""
    
    print(f"🚀 Iniciando análise completa para área: {area}")
    print("=" * 60)
    
    # Etapa 1: Gerar ideia
    print("\n1️⃣ GERANDO IDEIA...")
    ideia = chain_gerador.invoke({"area": area})
    print("💡 Ideia gerada:")
    print(ideia)
    
    # Etapa 2: Desenvolver ideia
    print("\n2️⃣ DESENVOLVENDO IDEIA...")
    proposta = chain_desenvolvedor.invoke({"ideia": ideia})
    print("📋 Proposta desenvolvida:")
    print(proposta)
    
    # Etapa 3: Analisar criticamente
    print("\n3️⃣ ANÁLISE CRÍTICA...")
    analise = chain_critico.invoke({"proposta": proposta})
    print("🔍 Análise final:")
    print(analise)
    
    return {
        "ideia_original": ideia,
        "proposta_desenvolvida": proposta,
        "analise_critica": analise
    }

# Testando nosso pipeline
resultado_completo = pipeline_completo_startup("sustentabilidade")

## 🔧 Chain Unificada - Tudo em Uma!

Tá, mas fazer isso manualmente é meio chato, né? Bora criar uma **Chain unificada** que faz tudo automaticamente!

**Dica!** Essa é a magia do LangChain: transformar processos manuais em pipelines automáticos! 🎪

In [ ]:
# Criando uma Chain unificada que faz todo o processo automaticamente
from langchain_core.runnables import RunnableLambda

# Função para organizar os resultados intermediários
def processar_sequencial(input_dict):
    area = input_dict["area"]
    
    # Etapa 1: Gerar ideia
    ideia = chain_gerador.invoke({"area": area})
    
    # Etapa 2: Desenvolver
    proposta = chain_desenvolvedor.invoke({"ideia": ideia})
    
    # Etapa 3: Analisar
    analise = chain_critico.invoke({"proposta": proposta})
    
    return {
        "area": area,
        "ideia": ideia,
        "proposta": proposta,
        "analise": analise
    }

# Criando nossa Chain unificada
chain_startup_completa = RunnableLambda(processar_sequencial)

print("🏗️ Chain unificada criada!")
print("Agora é só chamar chain_startup_completa.invoke({'area': 'sua_area'})")

# Teste rápido
resultado = chain_startup_completa.invoke({"area": "educação"})
print(f"\n✅ Processamento completo finalizado para área: {resultado['area']}")

## 📈 Monitoramento e Debugging de Chains

Quando suas Chains ficarem mais complexas (e elas vão ficar! 😅), você vai precisar debugar. Bora aprender as técnicas!

### Técnicas de Debug:
1. **Logging** - rastreamento de execução
2. **Timing** - medir performance
3. **Intermediate results** - resultados intermediários
4. **Error handling** - tratamento de erros

In [ ]:
# Sistema completo de debug para Chains
import time
from datetime import datetime

def chain_com_debug(area_teste):
    """Chain com sistema completo de debug e monitoramento"""
    
    debug_info = {
        "inicio": datetime.now(),
        "etapas": [],
        "tempos": [],
        "erros": []
    }
    
    print("🔍 EXECUTANDO CHAIN COM DEBUG COMPLETO")
    print("=" * 50)
    
    try:
        # Etapa 1: Geração
        print("\n1️⃣ Iniciando geração de ideia...")
        inicio_etapa = time.time()
        
        ideia = chain_gerador.invoke({"area": area_teste})
        
        tempo_etapa = time.time() - inicio_etapa
        debug_info["etapas"].append("Geração")
        debug_info["tempos"].append(tempo_etapa)
        
        print(f"   ✅ Concluído em {tempo_etapa:.2f}s")
        print(f"   📝 Preview: {ideia[:100]}...")
        
        # Etapa 2: Desenvolvimento
        print("\n2️⃣ Iniciando desenvolvimento...")
        inicio_etapa = time.time()
        
        proposta = chain_desenvolvedor.invoke({"ideia": ideia})
        
        tempo_etapa = time.time() - inicio_etapa
        debug_info["etapas"].append("Desenvolvimento")
        debug_info["tempos"].append(tempo_etapa)
        
        print(f"   ✅ Concluído em {tempo_etapa:.2f}s")
        print(f"   📝 Preview: {proposta[:100]}...")
        
        # Etapa 3: Análise
        print("\n3️⃣ Iniciando análise crítica...")
        inicio_etapa = time.time()
        
        analise = chain_critico.invoke({"proposta": proposta})
        
        tempo_etapa = time.time() - inicio_etapa
        debug_info["etapas"].append("Análise")
        debug_info["tempos"].append(tempo_etapa)
        
        print(f"   ✅ Concluído em {tempo_etapa:.2f}s")
        print(f"   📝 Preview: {analise[:100]}...")
        
    except Exception as e:
        debug_info["erros"].append(str(e))
        print(f"❌ Erro: {e}")
        return None, debug_info
    
    debug_info["fim"] = datetime.now()
    debug_info["tempo_total"] = sum(debug_info["tempos"])
    
    # Relatório final
    print("\n" + "=" * 50)
    print("📊 RELATÓRIO DE EXECUÇÃO")
    print("=" * 50)
    print(f"⏱️  Tempo total: {debug_info['tempo_total']:.2f}s")
    print(f"🔧 Etapas executadas: {len(debug_info['etapas'])}")
    print(f"❌ Erros encontrados: {len(debug_info['erros'])}")
    
    for i, (etapa, tempo) in enumerate(zip(debug_info["etapas"], debug_info["tempos"])):
        print(f"   {i+1}. {etapa}: {tempo:.2f}s")
    
    return {"ideia": ideia, "proposta": proposta, "analise": analise}, debug_info

# Testando nosso sistema de debug
resultado, debug = chain_com_debug("saúde digital")

## 📊 Visualização de Performance das Chains

Bora criar um gráfico pra visualizar a performance das nossas Chains!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dados do nosso debug anterior
if debug and debug["tempos"]:
    etapas = debug["etapas"]
    tempos = debug["tempos"]
    
    # Criando gráfico de barras
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico 1: Tempo por etapa
    cores = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    bars = ax1.bar(etapas, tempos, color=cores)
    ax1.set_title('⏱️ Tempo de Execução por Etapa', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Tempo (segundos)')
    ax1.set_xlabel('Etapas da Chain')
    
    # Adicionando valores nas barras
    for bar, tempo in zip(bars, tempos):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{tempo:.2f}s', ha='center', va='bottom', fontweight='bold')
    
    # Gráfico 2: Pizza da distribuição de tempo
    ax2.pie(tempos, labels=etapas, autopct='%1.1f%%', colors=cores, startangle=90)
    ax2.set_title('🥧 Distribuição de Tempo por Etapa', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Análise de Performance:")
    print(f"   Etapa mais rápida: {etapas[np.argmin(tempos)]} ({min(tempos):.2f}s)")
    print(f"   Etapa mais lenta: {etapas[np.argmax(tempos)]} ({max(tempos):.2f}s)")
    print(f"   Tempo médio por etapa: {np.mean(tempos):.2f}s")
else:
    print("⚠️ Dados de debug não disponíveis. Execute a célula anterior primeiro!")

## 🎯 Casos de Uso Práticos

Agora que você já manjou Chains, vamos ver alguns casos **super práticos** que você pode usar no dia a dia!

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-06_img_04.png)

### 1. Chain para Análise de Sentimentos + Sugestões

In [ ]:
# Chain prática: Análise de feedback + sugestão de ação

# Prompt para análise de sentimento
prompt_sentimento = ChatPromptTemplate.from_messages([
    ("system", "Analise o sentimento do texto: positivo, negativo ou neutro. Seja direto."),
    ("user", "Texto: {texto}")
])

# Prompt para sugestão de ação
prompt_acao = ChatPromptTemplate.from_messages([
    ("system", "Com base no sentimento '{sentimento}' do feedback, sugira 3 ações práticas."),
    ("user", "Feedback original: {texto_original}")
])

# Criando as chains
chain_sentimento = prompt_sentimento | model | StrOutputParser()
chain_acao = prompt_acao | model | StrOutputParser()

def analisar_feedback_completo(texto_feedback):
    """Analisa sentimento e sugere ações"""
    
    print(f"📝 Analisando feedback: {texto_feedback[:50]}...")
    
    # Etapa 1: Análise de sentimento
    sentimento = chain_sentimento.invoke({"texto": texto_feedback})
    print(f"\n😊 Sentimento detectado: {sentimento}")
    
    # Etapa 2: Sugestão de ações
    acoes = chain_acao.invoke({
        "sentimento": sentimento,
        "texto_original": texto_feedback
    })
    print(f"\n🎯 Ações sugeridas:")
    print(acoes)
    
    return {"sentimento": sentimento, "acoes": acoes}

# Testando com feedbacks reais
feedbacks_teste = [
    "O atendimento foi excelente! A equipe foi muito prestativa e resolveu meu problema rapidamente.",
    "Estou muito insatisfeito com o produto. Chegou com defeito e o suporte não responde.",
    "O produto é ok, funciona como esperado. Nada excepcional, mas atende."
]

for i, feedback in enumerate(feedbacks_teste, 1):
    print(f"\n" + "=" * 60)
    print(f"TESTE {i}")
    print("=" * 60)
    analisar_feedback_completo(feedback)

### 2. Chain para Geração de Conteúdo Otimizado

In [ ]:
# Chain prática: Gera conteúdo + otimiza para SEO + formata

# Chain 1: Gerador de conteúdo base
prompt_conteudo = ChatPromptTemplate.from_messages([
    ("system", "Você é um redator experiente. Escreva conteúdo engajante e informativo."),
    ("user", "Escreva um artigo de 200 palavras sobre: {topico}")
])

# Chain 2: Otimizador SEO
prompt_seo = ChatPromptTemplate.from_messages([
    ("system", "Otimize este conteúdo para SEO adicionando palavras-chave relevantes."),
    ("user", "Conteúdo original:\n{conteudo}\n\nPalavras-chave: {keywords}")
])

# Chain 3: Formatador final
prompt_formato = ChatPromptTemplate.from_messages([
    ("system", "Formate este conteúdo com markdown: títulos, subtítulos, listas, etc."),
    ("user", "Conteúdo para formatar:\n{conteudo_seo}")
])

# Criando as chains
chain_conteudo = prompt_conteudo | model | StrOutputParser()
chain_seo = prompt_seo | model | StrOutputParser()
chain_formato = prompt_formato | model | StrOutputParser()

def gerar_conteudo_completo(topico, palavras_chave):
    """Pipeline completo de geração de conteúdo"""
    
    print(f"✍️ Gerando conteúdo sobre: {topico}")
    print(f"🔍 Palavras-chave: {palavras_chave}")
    print("\n" + "=" * 50)
    
    # Etapa 1: Conteúdo base
    print("1️⃣ Gerando conteúdo base...")
    conteudo_base = chain_conteudo.invoke({"topico": topico})
    
    # Etapa 2: Otimização SEO
    print("2️⃣ Otimizando para SEO...")
    conteudo_seo = chain_seo.invoke({
        "conteudo": conteudo_base,
        "keywords": palavras_chave
    })
    
    # Etapa 3: Formatação final
    print("3️⃣ Aplicando formatação...")
    conteudo_final = chain_formato.invoke({"conteudo_seo": conteudo_seo})
    
    print("\n" + "=" * 50)
    print("📄 CONTEÚDO FINAL")
    print("=" * 50)
    print(conteudo_final)
    
    return conteudo_final

# Testando nosso gerador de conteúdo
conteudo = gerar_conteudo_completo(
    "Inteligência Artificial no Brasil",
    "IA, Brasil, tecnologia, inovação, futuro"
)

## 🏋️‍♂️ Exercícios Práticos

Agora é sua vez! Bora colocar a mão na massa com alguns exercícios práticos. 

**Dica!** Tente fazer primeiro, depois olha as soluções! 💪

### 🎯 Exercício 1: Chain de Tradução Multilíngue

**Desafio:** Crie uma Chain que:
1. Detecta o idioma do texto
2. Traduz para inglês (se não for inglês)
3. Traduz do inglês para português
4. Formata o resultado final

```python
# Sua solução aqui!
def criar_chain_traducao():
    # TODO: Implementar
    pass
```

In [ ]:
# 🎯 EXERCÍCIO 1 - Sua vez de implementar!
# Dica: use o padrão que aprendemos - prompt | model | parser

# Espaço para sua solução
print("💪 Implemente sua Chain de tradução aqui!")
print("Dica: Crie 4 prompts diferentes para cada etapa")

# Exemplo de estrutura:
# prompt_deteccao = ChatPromptTemplate.from_messages([...])
# prompt_traducao_ingles = ChatPromptTemplate.from_messages([...])
# ... e assim por diante

# Teste com textos em diferentes idiomas:
textos_teste = [
    "Hola, ¿cómo estás?",  # Espanhol
    "Bonjour, comment allez-vous?",  # Francês
    "Hello, how are you?",  # Inglês
    "Guten Tag, wie geht es Ihnen?"  # Alemão
]

### 🎯 Exercício 2: Chain de Análise de Produto

**Desafio:** Crie uma Chain que analisa reviews de produtos e:
1. Extrai pontos positivos e negativos
2. Atribui uma nota de 1 a 10
3. Gera um resumo executivo
4. Sugere melhorias para o produto

Use JSON como output final!

In [ ]:
# 🎯 EXERCÍCIO 2 - Chain de análise de produto

print("🔍 Crie sua Chain de análise de reviews aqui!")
print("\nReviews de teste:")

reviews_teste = [
    "Produto excelente! Chegou rápido, bem embalado. A qualidade superou minhas expectativas. Único ponto negativo é o preço um pouco alto, mas vale a pena.",
    "Muito decepcionado. O produto veio com defeito, o atendimento é péssimo e ainda não consegui resolver. Não recomendo.",
    "Produto mediano. Funciona como esperado, nada excepcional. Entrega foi ok. Para o preço está justo."
]

for i, review in enumerate(reviews_teste, 1):
    print(f"\nReview {i}: {review}")

# Sua implementação aqui!
# Dica: Use JsonOutputParser para estruturar o resultado

## 🎉 Soluções dos Exercícios

Agora vamos ver as soluções! Mas tenta fazer primeiro, tá? 😉

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-06_img_05.png)

In [ ]:
# 💡 SOLUÇÃO EXERCÍCIO 1: Chain de Tradução

# Prompts para cada etapa
prompt_deteccao = ChatPromptTemplate.from_messages([
    ("system", "Detecte o idioma do texto. Responda apenas o nome do idioma em português."),
    ("user", "Texto: {texto}")
])

prompt_para_ingles = ChatPromptTemplate.from_messages([
    ("system", "Traduza o texto para inglês. Se já estiver em inglês, retorne o texto original."),
    ("user", "Texto: {texto}")
])

prompt_para_portugues = ChatPromptTemplate.from_messages([
    ("system", "Traduza o texto do inglês para português brasileiro."),
    ("user", "Texto em inglês: {texto_ingles}")
])

# Chains individuais
chain_deteccao = prompt_deteccao | model | StrOutputParser()
chain_ingles = prompt_para_ingles | model | StrOutputParser()
chain_portugues = prompt_para_portugues | model | StrOutputParser()

def traduzir_multilíngue(texto):
    print(f"🌍 Traduzindo: {texto}")
    
    # 1. Detectar idioma
    idioma = chain_deteccao.invoke({"texto": texto})
    print(f"   📍 Idioma detectado: {idioma}")
    
    # 2. Traduzir para inglês
    texto_ingles = chain_ingles.invoke({"texto": texto})
    print(f"   🇺🇸 Em inglês: {texto_ingles}")
    
    # 3. Traduzir para português
    texto_portugues = chain_portugues.invoke({"texto_ingles": texto_ingles})
    print(f"   🇧🇷 Em português: {texto_portugues}")
    
    return {
        "original": texto,
        "idioma_original": idioma,
        "ingles": texto_ingles,
        "portugues": texto_portugues
    }

# Testando nossa solução
textos_teste = [
    "Hola, ¿cómo estás?",
    "Bonjour, comment allez-vous?"
]

for texto in textos_teste:
    print("\n" + "="*50)
    resultado = traduzir_multilíngue(texto)

print("\n✅ Solução do Exercício 1 concluída!")

In [ ]:
# 💡 SOLUÇÃO EXERCÍCIO 2: Chain de Análise de Produto

# Estrutura de dados para o resultado
class AnaliseReview(BaseModel):
    pontos_positivos: list = Field(description="Lista de pontos positivos")
    pontos_negativos: list = Field(description="Lista de pontos negativos")
    nota: int = Field(description="Nota de 1 a 10")
    resumo: str = Field(description="Resumo executivo")
    sugestoes: list = Field(description="Sugestões de melhoria")

# Parser JSON estruturado
parser_analise = JsonOutputParser(pydantic_object=AnaliseReview)

# Prompt para análise completa
prompt_analise_review = ChatPromptTemplate.from_messages([
    ("system", "Você é um analista de produtos experiente. Analise o review e retorne as informações estruturadas.\n{format_instructions}"),
    ("user", "Review para analisar: {review}")
])

# Chain completa
chain_analise_review = (
    {
        "review": RunnablePassthrough(),
        "format_instructions": lambda x: parser_analise.get_format_instructions()
    }
    | prompt_analise_review
    | model
    | parser_analise
)

def analisar_review_completo(review):
    """Análise completa de review com Chain estruturada"""
    print(f"🔍 Analisando review...")
    
    resultado = chain_analise_review.invoke(review)
    
    print("\n" + "="*60)
    print("📊 ANÁLISE COMPLETA DO REVIEW")
    print("="*60)
    
    print(f"\n✅ Pontos Positivos:")
    for ponto in resultado['pontos_positivos']:
        print(f"   • {ponto}")
    
    print(f"\n❌ Pontos Negativos:")
    for ponto in resultado['pontos_negativos']:
        print(f"   • {ponto}")
    
    print(f"\n⭐ Nota: {resultado['nota']}/10")
    
    print(f"\n📝 Resumo: {resultado['resumo']}")
    
    print(f"\n💡 Sugestões de Melhoria:")
    for sugestao in resultado['sugestoes']:
        print(f"   • {sugestao}")
    
    return resultado

# Testando com os reviews
reviews_teste = [
    "Produto excelente! Chegou rápido, bem embalado. A qualidade superou minhas expectativas. Único ponto negativo é o preço um pouco alto, mas vale a pena."
]

for review in reviews_teste[:1]:  # Testando com um review
    resultado = analisar_review_completo(review)

print("\n✅ Solução do Exercício 2 concluída!")

## 🎯 Preparando pro Próximo Módulo: Memory Systems

Liiindo! Agora você já manja de Chains! 🎉

Mas tá faltando uma coisa importante... **Memória!** 🧠

No próximo módulo (**Módulo 7: Memory Systems**), vamos aprender como fazer nossas Chains "lembrarem" de conversas anteriores. Imagina um chatbot que esquece tudo a cada mensagem? Seria meio esquisito, né? 😅

**Preview do que vem por aí:**
- ConversationBufferMemory
- ConversationSummaryMemory  
- Chains com memória persistente
- Chatbots que "lembram" do contexto



**Dica!** As Chains que criamos hoje vão ser a base pra tudo que vem pela frente! 🚀

## 📚 Resumo do Módulo: O que Aprendemos?

Neste módulo você virou um **expert em Chains**! Vamos recapitular:

### ✅ Conceitos Fundamentais:
- **O que são Chains**: Sequências de componentes conectados
- **LCEL (LangChain Expression Language)**: Usando o operador `|`
- **Composição**: PromptTemplate + ChatModel + OutputParser

### ✅ Tipos de Chains:
- **Chains simples**: Um fluxo linear
- **Chains estruturadas**: Com JSON parsing
- **Chains sequenciais**: Uma chamando a outra
- **Chains unificadas**: Tudo automatizado

### ✅ Técnicas Avançadas:
- **Debug e monitoramento**: Rastreamento completo
- **Performance analysis**: Medição de tempos
- **Error handling**: Tratamento de erros
- **Casos práticos**: Aplicações do mundo real

### 🎯 Fórmula do Sucesso:
$$Chain = PromptTemplate | ChatModel | OutputParser$$

### 🚀 Próximos Passos:
1. **Módulo 7**: Memory Systems (adicionar memória às Chains)
2. **Módulo 8**: Document Loading (trabalhar com documentos)
3. **Módulo 9**: RAG Implementation (Chains + conhecimento externo)

**Parabéns!** Você agora tem uma base sólida para construir aplicações incríveis com LangChain! 🎪

**Dica Final:** Pratique criando suas próprias Chains. A criatividade é o limite! ✨